In [ ]:
import tweepy
import json
from pymongo import MongoClient

####input your credentials here
consumer_key = 'XXX'
consumer_secret = 'XXX'
access_token = 'XXX'
access_token_secret = 'XXX'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

client = MongoClient(username='AdminRaul', password='sere200%')
db = client['Streaming']
collection = db['Hashtags']

# Hashtag List Madrid
TARGETS = ['#4M', '#4Mayo', '#Elecciones4M', '#EleccionesMadrid', '4M', '4Mayo', '#DebateTelemadrid', '#HazloXMadrid', '#VotaPSOE', '#MadridConMónica', '#PorLoQueDeVerdadImporta', '#ElMadridQueImporta'  '#YoConAyuso', '#VotaLibertad', '#ProtegeMadrid', '#VotaSeguro', '#QueHableLaMayoria', '#QueHableLaMayoria4M', '#MadrileñosPorEdmundoBal', '#EligeCentro']

# Hashtag List Andalusia
# TARGETS = ['#19J', '#19Junio', '#19JAndalucia', '#Elecciones19J', '19J', '19Junio', '#EleccionesAndaluzas', '#EleccionesAndalucia', '#EleccionesAndalucia2022', '#DebateRTVE', '#ElDebateDecisivo', '#DebateDecisivo', '#AndaluciaAvanza', '#AndalucíaAvanza', '#JuanmaPresidente', '#SiVotamosGanamos', '#AndalucíaQuiereMás', '#VotaPSOE', '#CambioReal', '#OlonaPresidenta', '#VotaMacarenazo', '#VotaMacarenaPresidenta', '#PorAndalucía', '#19JAdelante', '#Adelante19J', '#VotaTere19J', '#VotaAdelante19J', '#AdelanteAndalucia', '#EnDefensaPropia', '#AndalucíaEnDefensaPropia', '#AndalucíaLiberal', '#ElCambioQueFunciona']


print("Tracking: " + str(TARGETS))

In [ ]:
class StreamListener(tweepy.StreamListener):    

    def on_connect(self):
        print("You are now connected to the streaming API.")
 
    def on_error(self, status_code):
        print('An Error has occured: ' + repr(status_code))
        return False
 
    def on_data(self, data):      
        datajson = json.loads(data)
        text = datajson['text']
        fecha = datajson['created_at']      
        print("Tweet capturado a las "  + str(fecha) + " con el texto " + str(text))    
        collection.insert_one(datajson)

listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True, wait_on_rate_limit_notify=True, retry_count=10, retry_delay=5)) 
streamer = tweepy.Stream(auth=auth, listener=listener)

In [ ]:
from urllib3.exceptions import ProtocolError
from urllib3.exceptions import ReadTimeoutError

def start_stream(stream, **kwargs):
    try:
        stream.filter(**kwargs)
#    except KeyboardInterrupt:
#        sys.exit()
    except ReadTimeoutError:
        stream.disconnect()
        print("ReadTimeoutError exception")
        start_stream(stream, **kwargs)
    except ProtocolError:
        stream.disconnect()
        print("Protocol Error exception")
        start_stream(stream, **kwargs)
    except Exception as e:
        print(e)
        stream.disconnect()
        print("Fatal exception")
        start_stream(stream, **kwargs)
        
start_stream(streamer, track=TARGETS)